# Lab 12: Remote sensing of Snow

**Purpose:** The following notebook provides examples for processing snow information from remote sensing datasets. Students will walk through the algorithm to calculate snow cover from remote sensing data as well as a methodology that was developed to demonstrate how to produce annual maps representing the first day within a year where a given pixel reaches zero percent snow cover.

In [ ]:
%pylab inline

In [ ]:
# import ee api and geemap package
import ee
import math
import geemap
import pandas as pd
from geemap import colormaps as cmaps

In [ ]:
# try to initalize an ee session
# if not authenticated then run auth workflow and initialize
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

## Background

Snow is a very important process to hydrology. It has broad ecological implications and thus impacts human livelihoods, particularly in and around high latitude and mountainous systems.

One of the most important phases of the hydrologic cycle within these regions, the annual melting of accumulated winter snowfall provides the dominant source of water for streamflow and groundwater recharge for approximately one sixth of the global population

The anticipated warmer temperatures will alter the type and onset of precipitation; multiple regions, including the Rocky Mountains of North America have already measured a reduction in snowpack volume and warmer temperatures have shifted precipitation from snowfall to rain, causing snowmelt to occur earlier

This tutorial calculates the first day of no snow annually at the pixel level, providing the user with the ability to track the seasonal and interannual variability in the timing of snowmelt toward a better understanding of how the hydrological cycles of higher latitude and mountainous regions are responding to climate change.




## Calculating Snow Cover

Here we will apply the snow cover mapping algorithm to a Landsat image to exercise our image processing skills by calculating snow cover from scratch. Snow cover is calculated using a straighforward process of thresholds with some twists. First we will calculate some indices, apply thresholds from published research, and apply a relationship between vegetation and snow to get a more accurate snow cover estimate.

The process is taken from the MODIS snow product [Algorithm Theoretical Basis Document (ATBD)](https://modis-snow-ice.gsfc.nasa.gov/?c=atbd&t=atbd)

In [ ]:
# load in a landsat image for a snow area
pt = ee.Geometry.Point(-151.278028, 63.177048) # Denali National Park

img = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    # filter for Denali national park
    .filterBounds(pt)
     # filter for sep/oct, we want snow but not all snow
    .filter(ee.Filter.calendarRange(9,10,"month"))
    # sort by cloud cover, we want clear imager
    .sort("CLOUD_COVER")
    .first()
    # select only the reflectance bands
    .select("SR_B[1-7]")
    # rescale to reflectance values
    .multiply(0.0000275).add(-0.2)
)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(img,10)

Map.addLayer(img, {"bands":"SR_B4,SR_B3,SR_B2","min": 0.0, "max": 0.33, "gamma":1.3}, 'Landsat 8 VIS');
Map.addLayer(img, {"bands":"SR_B7,SR_B5,SR_B3","min": 0.05, "max": 0.55, "gamma":1.5}, 'Landsat 8');

Map.addLayerControl()

Map

The snow cover algorithm is largely based off of the Normalized Difference Snow Index (NDSI), however, we need to account for snow in forested areas (i.e. mixed pixels) so we will also calculate NDVI.

In [ ]:
# define function for calculating NDVI and NDSI for image
def calc_indices(img):
    ndsi = img.normalizedDifference(["SR_B3","SR_B6"]).rename("ndsi")
    ndvi = img.normalizedDifference(["SR_B5","SR_B4"]).rename("ndvi")
    return img.addBands(ndsi).addBands(ndvi)

# apply function to add indices
img_indices = calc_indices(img)

Set threshold values from section 4.2.1 of the ATBD:

In [ ]:
# greater than threshold, then snow
ndsi_threshold = 0.4

# used to seperate water vs snow
# snow reflects more in the nir band than water
nir_water_threshold = 0.11

# used to prevent pixels with very low visible reflectances
# for example black spruce stands
grn_dark_threshold = 0.1

In [ ]:
# apply thresholds to the appropriate bands
ndsi_mask = img_indices.select("ndsi").gt(ndsi_threshold)

nir_mask = img_indices.select("SR_B5").gt(nir_water_threshold)
grn_mask = img_indices.select("SR_B3").gt(grn_dark_threshold)

Now for the "fun" part, we need to create a mask based on the relationship of NDVI and NDSI. The equations for NDVI-NDSI mask were estimated from interpreting figure 6 from [Klein et al., 1998](https://doi.org/10.1002/(SICI)1099-1085(199808/09)12:10/11%3C1723::AID-HYP691%3E3.0.CO;2-2). This uses [ternary operations](https://developers.google.com/earth-engine/guides/image_relational#conditional-operators) to combine some arithmetric and conditional statements into one crazy expression.

In [ ]:
# create a mask from th
ndvi_ndsi_mask = img_indices.expression("(ndsi < 0.4 ? 1 : 0) & " +
                                        "(ndvi > (ndsi * -0.5 + 0.3) ? 1 : 0) & "+
                                        "(ndsi > (ndvi * -0.4 + 0.04) ? 1 : 0)",
    {
        "ndvi": img_indices.select("ndvi"),
        "ndsi": img_indices.select("ndsi")
    }
)

In [ ]:
# combine the masks
# use ndsi OR ndvi-ndsi because they do not overlap
# then apply AND for the other because they overlap
snow_mask = ndsi_mask.Or(ndvi_ndsi_mask).And(nir_mask).And(grn_mask)

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(img,10)

Map.addLayer(img_indices, {"bands":"SR_B7,SR_B5,SR_B3","min": 0.05, "max": 0.55, "gamma":1.5}, 'Landsat 8');
Map.addLayer(img_indices, {"bands":"ndsi","min": -0.1, "max": 1}, 'NDSI');
Map.addLayer(snow_mask.selfMask(), {"min": -1, "max": 1, "palette":"black,magenta"}, 'Snow Cover');
Map.addLayer(ndvi_ndsi_mask.selfMask(), {"min": -1, "max": 1, "palette":"black,limegreen"}, 'Veg Snow Cover');


Map.addLayerControl()

Map

The resulting snow mask is not the exact solution because of estimating the equations for the NDVI-NDSI relationship, however, this illustrates the process an implmentation using Earth Engine.

## Identifying the First Day of Zero Percent Snow Cover

This section covers building an ImageCollection where each image is a mosaic of pixels that describe the first day in a given year that zero percent snow cover is recorded. Snow cover is defined by the MODIS NDSI Snow Cover product.


### Set up our analysis

Here we are going to set up some variables for our process. First we define the day-of-year (DOY) to start the search for the first day with zero percent snow cover. For applications in the northern hemisphere, you will likely want to start with January 1st (DOY 1). However, if you are studying snowmelt timing in the southern hemisphere (e.g., the Andes), where snowmelt can occur on dates either side of the new year, it is more appropriate to start the year on July 1st (DOY 183), for instance.

In [ ]:
start_doy = 1

Define the year to start and end tracking snow cover fraction. All years in the range will be included in the analysis.

In [ ]:
start_year = 2000
end_year = 2021

years = ee.List.sequence(start_year, end_year)

Import the MODIS Snow Cover Daily Global 500m product and select the `NDSI_Snow_Cover` band.

Note: this is the collection we will be using to calculate date of snowmelt!

In [ ]:
# import collection
snow_collection = (
    ee.ImageCollection('MODIS/006/MOD10A1')
    .select('NDSI_Snow_Cover')
)

We do not apply any QA masking on our image collection...why?!

### Define an analysis mask

We did not apply a QA mask but we still need to constrain our analysis to relevant areas. This mask can be used to constrain the analysis to certain latitudes (`ee.Image.pixelLonLat()`), land cover types, geometries, etc. In this case we will: 1) mask out water so that the analysis is confined to pixels over landforms only; 2) mask out pixels that have very few days of snow cover; 3) mask out pixels that are snow covered for a good deal of the year (e.g., glaciers).



Import the MODIS water/land mask dataset, select the `water_mask` band, and set all land pixels to value 1:

In [ ]:
# Load in the MODIS water mask data
# set all land pixels to 1 and water pixels to 0
water_mask = (
    ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24')
    .select('water_mask')
    .Not()
)

Mask pixels based on frequency of snow cover:

In [ ]:
# function to convert 'NDSI_snow_cover' to binary mask
# uses a threshold of 10%
def is_snow(img):
    return img.gte(10)

# get the number of snow days
snow_days =  (
    snow_collection.filterDate('2018-01-01', '2019-01-01')
    .map(is_snow)
    .sum()
)

# Pixels must have been snow covered for at least 2 weeks in 2018
emphemeral_snow_cover = snow_days.gte(14)

# Pixels must not be snow covered more than 150 days in 2018.
const_snow_cover = snow_days.lte(150)

Combine the water mask and the snow cover frequency masks and apply mask to each image:

In [ ]:
analysis_mask = water_mask.And(emphemeral_snow_cover).And(const_snow_cover)

# snow_collection = snow_collection.map(lambda x: x.updateMask(analysis_mask))

In [ ]:
# get a mosaic for winter months
djf_mosaic = snow_collection.filter(ee.Filter.calendarRange(start=12,end=1,field="month")).mean()

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.addLayer(djf_mosaic, {"min": 0, "max": 100, "palette": cmaps.get_palette("inferno")}, 'Average DJF Snow');

Map.addLayerControl()

Map

### Identify the first day of the year without snow per pixel, per year

We want to calculate when each pixel goes to 0 for a given year. To do so we will implment the following steps for each year:

1. Define the start and end dates to filter the dataset for the given year.
2. Filter the image collection by the date range.
3. Add the date bands to each image in the filtered collection.
4. Sort the filtered collection by date. (Note: to determine the first day with snow accumulation in the fall, reverse sort the filtered collection.)
5. Make a mosaic using the min reducer to select the pixel with 0 (minimum) snow cover. Since the collection is sorted by date, the first image with 0 snow cover is selected. This operation is conducted per-pixel to build the complete image mosaic.
6. Apply the analysis mask to the resulting mosaic.


In [ ]:
def detect_no_snow(year):
    # function for setting date band information
    def add_date_bands(img):
        # Get image date.
        date = img.date();
        # Get calendar day-of-year.
        cal_doy = date.getRelative('day', 'year');
        # Get relative day-of-year; enumerate from user-defined startDoy.
        rel_doy = date.difference(start_date, 'day');
        # Get the date as milliseconds from Unix epoch.
        millis = date.millis();
        # Add all of the above date info as bands to the snow fraction image.
        date_bands = (
            ee.Image.constant([cal_doy, rel_doy, millis, year])
            .rename(['calDoy', 'relDoy', 'millis', 'year'])
        )
        # Cast bands to correct data type before returning the image.
        return (
            img.addBands(date_bands)
            .cast({'calDoy': 'float', 'relDoy': 'float', 'millis': 'long','year': 'int'})
            .set('millis', millis)
        )

    # Get the first day-of-year for this year as an ee.Date object.
    first_doy = ee.Date.fromYMD(year, 1, 1)

    # Advance from the firstDoy to the user-defined startDay; subtract 1 since
    # firstDoy is already 1. Set the result as the global startDate variable so
    # that it is accessible to the addDateBands mapped to the collection below.
    start_date = first_doy.advance(start_doy-1, 'day');
    # Get endDate for this year by advancing 1 year from startDate.
    # Need to advance an extra day because end date of filterDate() function
    # is exclusive.
    end_date = start_date.advance(1, 'year').advance(1, 'day');
    
    # Filter the complete collection by the start and end dates just defined.
    year_col = snow_collection.filterDate(start_date, end_date)


    # Construct an image where pixels represent the first day within the date
    # range that the lowest snow fraction is observed.
    no_snow_img = (
        year_col
        .map(add_date_bands)
        # Sort the images by ascending time to identify the first day without
        # snow. Alternatively, you can use .sort('millis', false) to
        # reverse sort (find first day of snow in the fall).
        .sort('millis')
        # Make a mosaic composed of pixels from images that represent the
        # observation with the minimum percent snow cover (defined by the
        # NDSI_Snow_Cover band); include all associated bands for the selected
        # image.
        .reduce(ee.Reducer.min(5))
        # Rename the bands - band names were altered by previous operation.
        .rename(['snowCover', 'calDoy', 'relDoy', 'millis', 'year'])
        # Set the year as a property for filtering by later.
        .set('year', year)
        # Apply the mask.
        .updateMask(analysis_mask)
    )

    # Mask by minimum snow fraction - only include pixels that reach 0
    # percent cover. Return the resulting image.
    return no_snow_img.updateMask(no_snow_img.select('snowCover').eq(0))


In [ ]:
# apply algorithm
annual_no_snow = ee.ImageCollection.fromImages(years.map(detect_no_snow))

Next we will filter a single year (2019 in the example below) from the collection and display the image to the Map to see spatial patterns of snowmelt timing

In [ ]:
# set year of interest to display
year_of_interest = 2019

In [ ]:
# extract out the image we want
first_day_no_snow = annual_no_snow.filter(ee.Filter.eq('year', year_of_interest)).first()

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.addLayer(first_day_no_snow , {"bands":"calDoy","min": 0, "max": 200, "palette": cmaps.get_palette("viridis_r")}, f'First day of no snow, {year_of_interest}');

Map.addLayerControl()

Map.add_colorbar({"bands":"calDoy","min": 0, "max": 200, "palette": cmaps.get_palette("viridis_r")}, label='First day of no snow')

Map

### Time series calculation

To visually understand the temporal patterns of the first date of no snow through time, we can display our results in a time series chart. In this case we will see if the first day without snow changes in time for Utah. We are also going to go one step further and calculate this trend for different elevation areas:

In [ ]:
# load in the UT state feature
ut = (
    ee.FeatureCollection("TIGER/2018/States")
    .filter(ee.Filter.eq("NAME","Utah"))
)

In [ ]:
# load in a DEM image
elv = ee.Image("NASA/NASADEM_HGT/001").select("elevation")

In [ ]:
# specify number of elevation bands to create
n_bands = ee.List.sequence(0,5)
# set starting height
base_band = ee.Number(500)
# set interval for band
band_height = ee.Number(500)

In [ ]:
# define a function to calculate 
def calc_band(i):
    return ee.Image.constant(0).where(elv.gt(base_band.add(band_height.multiply(i))),1)

elv_bands = ee.ImageCollection.fromImages(n_bands.map(calc_band)).sum().clip(ut.geometry())

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(ut,7)

Map.addLayer(ut, {}, 'Utah');
Map.addLayer(elv_bands, {"min":1,"max":6,"palette":cmaps.get_palette("gist_earth")}, 'Elevation bands');

Map.addLayerControl()

Map

In [ ]:
def avg_no_snow(i):
    def avg_band(img):
        # apply reduction to get the average for the elevation band of interest
        avg = img.select("calDoy").updateMask(elv_band_mask).reduceRegion(
            geometry = ut.geometry(1e3).bounds(1e3),
            reducer = ee.Reducer.mean(),
            scale = 500,
            bestEffort=True,
            tileScale=4
        )
        return img.set(avg)

    # get which elevation band we are working on
    i = ee.Number(i)
    elv_band_mask = elv_bands.eq(i.add(1))

    # apply reductions
    val_col = annual_no_snow.map(avg_band)
    # return only the list as we will combine them
    return val_col.aggregate_array("calDoy")


# wrap multiple requests in a loop
# this will help prevent the dreaded "Too many concurrent aggregations" error
snow_list = []
for i in range(1,6):
    i_band_avg = avg_no_snow(i) # apply function
    snow_list.append(i_band_avg.getInfo()) # append results


In [ ]:
# get a list of bands and the bottom elevation value
band_names = [f"band_{500+(i*500)}" for i in range(1,6)]

# create a dictionary from the lists and band names
df_dict = {bname: snow_list[i] for i,bname in enumerate(band_names)}

In [ ]:
# create a dataframe from the time series information
df = pd.DataFrame(df_dict, index=years.getInfo())

In [ ]:
df.plot(figsize=(10,5),marker="o");

# Assignment

1. Often times it is useful to compare the change from a reference year. For this question, we will compare year-to-year difference in melt timing by selecting two years of interest from the collection and subtracting them. Calculate the difference between 2013 and 2012 and display the results on a map.

2. It is also possible to identify trends in the shifting first DOY with no snow by calculating the slope through a pixel’s time series points. For this quesitons, apply a Sen's Slope reducer (i.e. `ee.Reducer.sensSlope()`) to calculate the annual trend and display the trend (i.e. slope band) on a map. All of the bands required should be calculated for the images. Note: This exercise is about implementing the trend analysis. Goodness of fit is not measured here, nor is significance considered. Inter-annual variability can be influence the slope too.

In [ ]:
years = ee.List.sequence(start_year, end_year)

In [ ]:
# annual_no_snow

In [ ]:
def add_time_band(img):
    yr = ee.Number(img.get('year'))
    yr_dt = ee.Date.fromYMD(yr, 1,1)
    time_off = ee.Date("1970-01-01").difference(yr_dt, "year")
    # yr_img = ee.Image.constant(yr).float().rename("time")
    yr_img = ee.Image.constant(time_off).float().rename("time")
    return img.addBands(yr_img)


annual_no_snow_time = annual_no_snow.map(add_time_band)

In [ ]:
trend = annual_no_snow_time.select(["time","calDoy"]).reduce(ee.Reducer.sensSlope())

In [ ]:
# Visualize the results
Map = geemap.Map()

Map.centerObject(ut,7)

Map.addLayer(ut, {}, 'Utah');
Map.addLayer(trend, {"bands":"slope", "min":-5,"max":5,"palette":cmaps.get_palette("bwr")}, 'Day of snowmelt trend');

Map.addLayerControl()

Map